1. Advanced Oil Spill Analysis with Economic Impact

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# Set professional style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")

# Load and prepare data
df = pd.read_csv('oil-spill.csv')
df['Total_Spills'] = df['Large oil spills (>700 tonnes)'] + df['Medium oil spills (7-700 tonnes)']
df['Spill_Severity_Index'] = (df['Large oil spills (>700 tonnes)'] * 10 + 
                             df['Medium oil spills (7-700 tonnes)'] * 1)

# Economic impact assumptions (in millions USD)
LARGE_SPILL_COST = 50  # Average cost per large spill
MEDIUM_SPILL_COST = 5  # Average cost per medium spill
df['Economic_Impact'] = (df['Large oil spills (>700 tonnes)'] * LARGE_SPILL_COST +
                        df['Medium oil spills (7-700 tonnes)'] * MEDIUM_SPILL_COST)

# Time series decomposition
decomposition = seasonal_decompose(df.set_index('Year')['Total_Spills'], 
                                  period=5, model='additive')

# Trend analysis with linear regression
X = df['Year'].values.reshape(-1, 1)
y = df['Total_Spills'].values
model = LinearRegression()
model.fit(X, y)
trend_line = model.predict(X)

# Create comprehensive visualization
fig, axes = plt.subplots(3, 2, figsize=(16, 14))
fig.suptitle('Comprehensive Oil Spill Analysis (1970-2022)', fontsize=16, fontweight='bold')

# Time series plots
axes[0, 0].plot(df['Year'], df['Total_Spills'], marker='o', linewidth=2, label='Total Spills')
axes[0, 0].plot(df['Year'], trend_line, 'r--', linewidth=2, label='Trend Line')
axes[0, 0].set_title('Oil Spills Over Time with Trend')
axes[0, 0].set_ylabel('Number of Spills')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Economic impact
axes[0, 1].bar(df['Year'], df['Economic_Impact'], alpha=0.7, color='orange')
axes[0, 1].set_title('Estimated Economic Impact of Spills')
axes[0, 1].set_ylabel('Cost (Million USD)')
axes[0, 1].tick_params(axis='x', rotation=45)

# Decomposition plots
axes[1, 0].plot(df['Year'], decomposition.trend, linewidth=2)
axes[1, 0].set_title('Trend Component')
axes[1, 0].set_ylabel('Spills')

axes[1, 1].plot(df['Year'], decomposition.seasonal, linewidth=2)
axes[1, 1].set_title('Seasonal Component')
axes[1, 1].set_ylabel('Spills')

# Comparative analysis by decade
df['Decade'] = (df['Year'] // 10) * 10
decade_stats = df.groupby('Decade').agg({
    'Large oil spills (>700 tonnes)': 'sum',
    'Medium oil spills (7-700 tonnes)': 'sum',
    'Total_Spills': 'sum',
    'Economic_Impact': 'sum'
}).reset_index()

axes[2, 0].bar(decade_stats['Decade'], decade_stats['Total_Spills'], 
               alpha=0.7, color='green')
axes[2, 0].set_title('Total Spills by Decade')
axes[2, 0].set_ylabel('Number of Spills')
axes[2, 0].set_xlabel('Decade')

# Economic impact by decade
axes[2, 1].bar(decade_stats['Decade'], decade_stats['Economic_Impact'], 
               alpha=0.7, color='red')
axes[2, 1].set_title('Economic Impact by Decade')
axes[2, 1].set_ylabel('Cost (Million USD)')
axes[2, 1].set_xlabel('Decade')

plt.tight_layout()
plt.show()

# Statistical analysis
print("="*60)
print("STATISTICAL ANALYSIS OF OIL SPILL DATA")
print("="*60)

# Calculate percentage changes
first_5yr_avg = df[df['Year'] <= 1974]['Total_Spills'].mean()
last_5yr_avg = df[df['Year'] >= 2018]['Total_Spills'].mean()
pct_change = ((first_5yr_avg - last_5yr_avg) / first_5yr_avg) * 100

print(f"Average annual spills (1970-1974): {first_5yr_avg:.2f}")
print(f"Average annual spills (2018-2022): {last_5yr_avg:.2f}")
print(f"Percentage reduction: {pct_change:.2f}%")

# Trend statistical significance
slope, intercept, r_value, p_value, std_err = stats.linregress(df['Year'], df['Total_Spills'])
print(f"\nTrend analysis - Slope: {slope:.4f} (spills per year)")
print(f"P-value: {p_value:.4f} ({'Statistically significant' if p_value < 0.05 else 'Not statistically significant'})")

# Risk assessment
mean_spills = df['Total_Spills'].mean()
std_spills = df['Total_Spills'].std()
risk_threshold = mean_spills + std_spills

high_risk_years = df[df['Total_Spills'] > risk_threshold]['Year'].tolist()
print(f"\nHigh-risk years (above {risk_threshold:.1f} spills): {high_risk_years}")

# Economic summary
total_economic_impact = df['Economic_Impact'].sum()
print(f"\nTotal estimated economic impact (1970-2022): ${total_economic_impact:,.2f} million")

# Save detailed report
report_data = {
    'Metric': [
        'Total Large Spills (1970-2022)',
        'Total Medium Spills (1970-2022)',
        'Combined Total Spills',
        'Total Economic Impact (Million USD)',
        'Average Annual Spills (1970-1974)',
        'Average Annual Spills (2018-2022)',
        'Percentage Reduction',
        'Trend Slope (spills/year)',
        'Trend Statistical Significance (p-value)',
        'High-Risk Years (above mean + 1 std)'
    ],
    'Value': [
        df['Large oil spills (>700 tonnes)'].sum(),
        df['Medium oil spills (7-700 tonnes)'].sum(),
        df['Total_Spills'].sum(),
        total_economic_impact,
        first_5yr_avg,
        last_5yr_avg,
        f"{pct_change:.2f}%",
        f"{slope:.4f}",
        f"{p_value:.4f}",
        ', '.join(map(str, high_risk_years))
    ]
}

report_df = pd.DataFrame(report_data)
report_df.to_csv('oil_spill_detailed_report.csv', index=False)
print("\nDetailed report saved to 'oil_spill_detailed_report.csv'")

2. Well Log Analysis and Petrophysics

In [ ]:
import lasio
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
import warnings
warnings.filterwarnings('ignore')

# Create synthetic well log data (in real scenarios, you would load from LAS files)
def create_synthetic_logs():
    depth = np.arange(1000, 3000, 0.5)  # Depth in meters
    gr = 40 + 100 * np.exp(-(depth - 2000)**2 / 100000) + np.random.normal(0, 5, len(depth))
    rt = 20 + 80 * np.exp(-(depth - 2200)**2 / 80000) + np.random.normal(0, 2, len(depth))
    nphi = 0.3 - 0.2 * np.exp(-(depth - 2100)**2 / 90000) + np.random.normal(0, 0.02, len(depth))
    rhob = 2.0 + 0.8 * np.exp(-(depth - 1900)**2 / 70000) + np.random.normal(0, 0.05, len(depth))
    
    # Create a DataFrame
    df = pd.DataFrame({
        'DEPTH': depth,
        'GR': gr,
        'RT': rt,
        'NPHI': nphi,
        'RHOB': rhob
    })
    
    # Add some lithology flags based on log responses
    df['LITHOLOGY'] = 'SHALE'
    df.loc[(df.GR < 70) & (df.RT > 30), 'LITHOLOGY'] = 'SANDSTONE'
    df.loc[(df.GR < 50) & (df.NPHI > 0.25) & (df.RHOB < 2.4), 'LITHOLOGY'] = 'SANDSTONE_POROUS'
    df.loc[(df.GR > 100) & (df.RT < 20), 'LITHOLOGY'] = 'SHALE'
    df.loc[(df.RHOB > 2.6) & (df.NPHI < 0.1), 'LITHOLOGY'] = 'LIMESTONE'
    
    return df

# Create synthetic data
well_df = create_synthetic_logs()

# Petrophysical calculations
def calculate_vshale(gr, gr_min=40, gr_max=120):
    """Calculate shale volume from gamma ray"""
    vshale = (gr - gr_min) / (gr_max - gr_min)
    return np.clip(vshale, 0, 1)

def calculate_porosity(nphi, rhob, nphi_matrix=0.01, rhob_matrix=2.65, rhob_fluid=1.0):
    """Calculate porosity from neutron and density logs"""
    # Density porosity
    phid = (rhob_matrix - rhob) / (rhob_matrix - rhob_fluid)
    
    # Neutron porosity
    phin = nphi
    
    # Average porosity
    porosity = (phid + phin) / 2
    
    return np.clip(porosity, 0, 0.4)

def calculate_sw(rt, porosity, a=0.81, m=2, n=2, rw=0.1):
    """Calculate water saturation using Archie's equation"""
    f = a / (porosity ** m)
    sw = (f * rw / rt) ** (1/n)
    return np.clip(sw, 0.1, 1)

# Apply calculations
well_df['VSHALE'] = calculate_vshale(well_df['GR'])
well_df['POROSITY'] = calculate_porosity(well_df['NPHI'], well_df['RHOB'])
well_df['SW'] = calculate_sw(well_df['RT'], well_df['POROSITY'])

# Create a comprehensive well log plot
def plot_well_logs(df, depth_range=None):
    if depth_range:
        df = df[(df.DEPTH >= depth_range[0]) & (df.DEPTH <= depth_range[1])]
    
    fig, axes = plt.subplots(1, 6, figsize=(15, 12))
    fig.suptitle('Well Log Analysis with Petrophysical Calculations', fontsize=16)
    
    # Track 1: Gamma Ray and VSHALE
    ax1 = axes[0]
    ax1.plot(df['GR'], df['DEPTH'], color='green', linewidth=0.5)
    ax1.set_xlabel('Gamma Ray (API)')
    ax1.set_ylabel('Depth (m)')
    ax1.xaxis.tick_top()
    ax1.xaxis.set_label_position('top')
    ax1.set_xlim(0, 150)
    ax1.grid(True)
    
    ax1b = ax1.twiny()
    ax1b.plot(df['VSHALE'], df['DEPTH'], color='red', linewidth=1)
    ax1b.set_xlabel('VShale', color='red')
    ax1b.tick_params(axis='x', labelcolor='red')
    ax1b.set_xlim(0, 1)
    
    # Track 2: Resistivity
    ax2 = axes[1]
    ax2.plot(df['RT'], df['DEPTH'], color='blue', linewidth=0.5)
    ax2.set_xlabel('Resistivity (ohm-m)')
    ax2.xaxis.tick_top()
    ax2.xaxis.set_label_position('top')
    ax2.set_xscale('log')
    ax2.set_xlim(0.2, 200)
    ax2.grid(True)
    
    # Track 3: Porosity
    ax3 = axes[2]
    ax3.plot(df['NPHI'], df['DEPTH'], color='red', linewidth=0.5, label='NPHI')
    ax3.plot(df['POROSITY'], df['DEPTH'], color='black', linewidth=1.5, label='Avg Porosity')
    ax3.set_xlabel('Porosity (v/v)')
    ax3.xaxis.tick_top()
    ax3.xaxis.set_label_position('top')
    ax3.set_xlim(0, 0.4)
    ax3.legend(loc='upper center')
    ax3.grid(True)
    
    # Track 4: Density
    ax4 = axes[3]
    ax4.plot(df['RHOB'], df['DEPTH'], color='blue', linewidth=0.5)
    ax4.set_xlabel('Density (g/cc)')
    ax4.xaxis.tick_top()
    ax4.xaxis.set_label_position('top')
    ax4.set_xlim(1.8, 2.8)
    ax4.grid(True)
    
    # Track 5: Water Saturation
    ax5 = axes[4]
    ax5.plot(df['SW'], df['DEPTH'], color='purple', linewidth=0.5)
    ax5.set_xlabel('Water Saturation')
    ax5.xaxis.tick_top()
    ax5.xaxis.set_label_position('top')
    ax5.set_xlim(0, 1)
    ax5.grid(True)
    
    # Track 6: Lithology
    ax6 = axes[5]
    colors = {'SHALE': 'gray', 'SANDSTONE': 'yellow', 
              'SANDSTONE_POROUS': 'orange', 'LIMESTONE': 'blue'}
    
    for lith, color in colors.items():
        lith_data = df[df['LITHOLOGY'] == lith]
        if not lith_data.empty:
            ax6.scatter([0] * len(lith_data), lith_data['DEPTH'], 
                       color=color, s=10, label=lith)
    
    ax6.set_xlabel('Lithology')
    ax6.xaxis.tick_top()
    ax6.xaxis.set_label_position('top')
    ax6.set_xlim(-1, 1)
    ax6.legend(loc='upper center')
    ax6.set_xticks([])
    
    for ax in axes:
        ax.set_ylim(df['DEPTH'].max(), df['DEPTH'].min())
        ax.grid(True)
    
    plt.tight_layout()
    plt.show()

# Plot the well logs for a specific interval
plot_well_logs(well_df, depth_range=(1800, 2300))

# Calculate net pay
well_df['NET_PAY'] = 0
well_df.loc[(well_df['POROSITY'] > 0.1) & (well_df['SW'] < 0.6) & 
           (well_df['VSHALE'] < 0.4), 'NET_PAY'] = 0.5  # 0.5m per sample

net_pay = well_df['NET_PAY'].sum()
print(f"Net pay: {net_pay:.1f} meters")

# Calculate hydrocarbon pore volume
well_df['HCPV'] = well_df['POROSITY'] * (1 - well_df['SW']) * well_df['NET_PAY']
hcpv = well_df['HCPV'].sum()
print(f"Hydrocarbon pore volume: {hcpv:.2f} m³/m²")

# Save results to CSV
well_df.to_csv('well_log_analysis_results.csv', index=False)
print("Well log analysis results saved to 'well_log_analysis_results.csv'")

3. Production Data Analysis and Forecasting

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from datetime import datetime, timedelta

# Create synthetic production data
def generate_production_data():
    # Generate dates
    start_date = datetime(2015, 1, 1)
    dates = [start_date + timedelta(days=30*i) for i in range(100)]
    
    # Generate production data with decline
    time = np.arange(0, 100)
    initial_rate = 1000  # barrels per month
    decline_rate = 0.03  # 3% monthly decline
    
    # Arps exponential decline
    production = initial_rate * np.exp(-decline_rate * time)
    
    # Add some noise
    production = production * np.random.normal(1, 0.1, len(time))
    
    # Add water production (increasing over time)
    water_cut = 0.1 + 0.8 * (time / 100)  # Water cut increases from 10% to 90%
    water = production * water_cut
    oil = production * (1 - water_cut)
    
    # Create DataFrame
    df = pd.DataFrame({
        'Date': dates,
        'Time': time,
        'Liquid_Rate': production,
        'Oil_Rate': oil,
        'Water_Rate': water,
        'Water_Cut': water_cut
    })
    
    return df

# Generate production data
production_df = generate_production_data()

# Define decline curve models
def exponential_decline(t, qi, di):
    """Exponential decline model"""
    return qi * np.exp(-di * t)

def hyperbolic_decline(t, qi, di, b):
    """Hyperbolic decline model"""
    return qi / (1 + b * di * t) ** (1/b)

def harmonic_decline(t, qi, di):
    """Harmonic decline model (special case of hyperbolic with b=1)"""
    return qi / (1 + di * t)

# Fit decline curves to oil production
t = production_df['Time'].values
q = production_df['Oil_Rate'].values

# Fit exponential decline
popt_exp, pcov_exp = curve_fit(exponential_decline, t, q, p0=[1000, 0.03])
q_exp = exponential_decline(t, *popt_exp)

# Fit hyperbolic decline
popt_hyp, pcov_hyp = curve_fit(hyperbolic_decline, t, q, p0=[1000, 0.03, 0.5], maxfev=5000)
q_hyp = hyperbolic_decline(t, *popt_hyp)

# Calculate EUR (Estimated Ultimate Recovery)
def calculate_eur(decline_func, params, t_max=300, dt=1):
    """Calculate EUR by integrating decline curve"""
    t_eval = np.arange(0, t_max, dt)
    q_eval = decline_func(t_eval, *params)
    eur = np.trapz(q_eval, dx=dt)
    return eur

eur_exp = calculate_eur(exponential_decline, popt_exp)
eur_hyp = calculate_eur(hyperbolic_decline, popt_hyp)

# Forecast future production
forecast_time = np.arange(0, 200, 1)
forecast_exp = exponential_decline(forecast_time, *popt_exp)
forecast_hyp = hyperbolic_decline(forecast_time, *popt_hyp)

# Create production analysis plot
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Production Data Analysis and Forecasting', fontsize=16)

# Production history and forecast
axes[0, 0].plot(production_df['Date'], production_df['Oil_Rate'], 
                'bo-', label='Historical Data', markersize=4)
axes[0, 0].plot(production_df['Date'].iloc[-1] + pd.to_timedelta(forecast_time*30, unit='D'), 
                forecast_exp, 'r--', label='Exponential Forecast')
axes[0, 0].plot(production_df['Date'].iloc[-1] + pd.to_timedelta(forecast_time*30, unit='D'), 
                forecast_hyp, 'g--', label='Hyperbolic Forecast')
axes[0, 0].set_title('Oil Production Rate')
axes[0, 0].set_ylabel('Oil Rate (bbl/month)')
axes[0, 0].legend()
axes[0, 0].grid(True)
axes[0, 0].tick_params(axis='x', rotation=45)

# Water cut analysis
axes[0, 1].plot(production_df['Date'], production_df['Water_Cut']*100, 'ro-', markersize=4)
axes[0, 1].set_title('Water Cut Trend')
axes[0, 1].set_ylabel('Water Cut (%)')
axes[0, 1].grid(True)
axes[0, 1].tick_params(axis='x', rotation=45)

# Decline curve analysis parameters
models = ['Exponential', 'Hyperbolic']
qi_values = [popt_exp[0], popt_hyp[0]]
di_values = [popt_exp[1], popt_hyp[1]]
b_values = [0, popt_hyp[2]]  # b=0 for exponential
eur_values = [eur_exp, eur_hyp]

x = np.arange(len(models))
width = 0.2

axes[1, 0].bar(x - width, qi_values, width, label='Initial Rate (Qi)')
axes[1, 0].bar(x, di_values, width, label='Decline Rate (Di)')
axes[1, 0].bar(x + width, b_values, width, label='B-factor')
axes[1, 0].set_title('Decline Curve Parameters')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(models)
axes[1, 0].legend()
axes[1, 0].grid(True, axis='y')

# EUR comparison
axes[1, 1].bar(models, eur_values, color=['red', 'green'])
axes[1, 1].set_title('Estimated Ultimate Recovery (EUR)')
axes[1, 1].set_ylabel('EUR (bbl)')
axes[1, 1].grid(True, axis='y')

plt.tight_layout()
plt.show()

# Print analysis results
print("="*60)
print("PRODUCTION ANALYSIS RESULTS")
print("="*60)
print(f"Exponential decline parameters: Qi = {popt_exp[0]:.1f} bbl/month, Di = {popt_exp[1]:.4f} 1/month")
print(f"Hyperbolic decline parameters: Qi = {popt_hyp[0]:.1f} bbl/month, Di = {popt_hyp[1]:.4f} 1/month, b = {popt_hyp[2]:.2f}")
print(f"Exponential EUR: {eur_exp:,.0f} bbl")
print(f"Hyperbolic EUR: {eur_hyp:,.0f} bbl")
print(f"Current water cut: {production_df['Water_Cut'].iloc[-1]*100:.1f}%")

# Calculate cumulative production
production_df['Cumulative_Oil'] = production_df['Oil_Rate'].cumsum()
print(f"Cumulative production: {production_df['Cumulative_Oil'].iloc[-1]:,.0f} bbl")

# Save production analysis results
production_df.to_csv('production_analysis_results.csv', index=False)
print("Production analysis results saved to 'production_analysis_results.csv'")

4. Economic Analysis and Investment Decision Modeling

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Oil price scenarios (USD/bbl)
oil_prices = {
    'Low': 50,
    'Base': 70,
    'High': 90
}

# Development cost scenarios (Million USD)
development_costs = {
    'Low': 150,
    'Base': 200,
    'High': 250
}

# Production profile (from previous analysis)
# Using the exponential decline forecast from previous example
time = np.arange(0, 120)  # 10 years monthly
qi = popt_exp[0]  # Initial rate from previous analysis
di = popt_exp[1]  # Decline rate from previous analysis
production = exponential_decline(time, qi, di)

# Economic parameters
opex_per_bbl = 15  # USD/bbl
royalties = 0.125  # 12.5%
tax_rate = 0.35    # 35%
discount_rate = 0.1  # 10%

# Economic model
def economic_model(oil_price, development_cost, production_profile):
    """Calculate economic metrics for a development scenario"""
    
    # Revenue calculation
    revenue = production_profile * oil_price
    
    # Cost calculation
    opex = production_profile * opex_per_bbl
    royalties_payment = revenue * royalties
    
    # Cash flow before tax
    cash_flow_before_tax = revenue - opex - royalties_payment
    
    # Depreciation (straight line over 5 years)
    depreciation = np.zeros_like(cash_flow_before_tax)
    depreciation[:60] = development_cost / 5 / 12  # Monthly depreciation
    
    # Taxable income
    taxable_income = cash_flow_before_tax - depreciation
    taxable_income = np.clip(taxable_income, 0, None)  # No negative taxation
    
    # Tax payment
    tax_payment = taxable_income * tax_rate
    
    # Net cash flow
    net_cash_flow = cash_flow_before_tax - tax_payment
    
    # Add development cost as negative cash flow in month 0
    net_cash_flow[0] -= development_cost
    
    # Calculate NPV
    discount_factors = 1 / (1 + discount_rate/12) ** time
    npv = np.sum(net_cash_flow * discount_factors)
    
    # Calculate IRR (simplified)
    def npv_func(rate):
        factors = 1 / (1 + rate/12) ** time
        return np.abs(np.sum(net_cash_flow * factors))
    
    result = minimize(npv_func, x0=0.1, method='Nelder-Mead')
    irr = result.x[0] if result.success else np.nan
    
    # Calculate payback period
    cumulative_cash_flow = np.cumsum(net_cash_flow)
    payback_period = np.argmax(cumulative_cash_flow >= 0) / 12 if np.any(cumulative_cash_flow >= 0) else np.nan
    
    return {
        'npv': npv,
        'irr': irr,
        'payback_period': payback_period,
        'net_cash_flow': net_cash_flow,
        'cumulative_cash_flow': cumulative_cash_flow
    }

# Run economic analysis for all scenarios
results = {}
for price_scenario, oil_price in oil_prices.items():
    for cost_scenario, development_cost in development_costs.items():
        scenario_name = f"Price_{price_scenario}_Cost_{cost_scenario}"
        results[scenario_name] = economic_model(oil_price, development_cost, production)

# Create economic analysis dashboard
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Economic Analysis of Oil Development Project', fontsize=16)

# NPV comparison
scenarios = list(results.keys())
npv_values = [results[s]['npv'] for s in scenarios]

axes[0, 0].barh(scenarios, npv_values, color=['green' if x >= 0 else 'red' for x in npv_values])
axes[0, 0].axvline(x=0, color='black', linestyle='--')
axes[0, 0].set_title('Net Present Value (NPV) by Scenario')
axes[0, 0].set_xlabel('NPV (Million USD)')

# IRR comparison
irr_values = [results[s]['irr'] for s in scenarios]

axes[0, 1].barh(scenarios, irr_values, color=['green' if x >= discount_rate else 'red' for x in irr_values])
axes[0, 1].axvline(x=discount_rate, color='black', linestyle='--', label='Hurdle Rate')
axes[0, 1].set_title('Internal Rate of Return (IRR) by Scenario')
axes[0, 1].set_xlabel('IRR')
axes[0, 1].legend()

# Cash flow profile for base case
base_case = results['Price_Base_Cost_Base']
months = np.arange(len(base_case['net_cash_flow'])) / 12

axes[1, 0].plot(months, base_case['net_cash_flow'] / 1000, label='Monthly Cash Flow')
axes[1, 0].plot(months, base_case['cumulative_cash_flow'] / 1000, label='Cumulative Cash Flow', linewidth=2)
axes[1, 0].axhline(y=0, color='black', linestyle='-')
axes[1, 0].set_title('Base Case Cash Flow Profile')
axes[1, 0].set_xlabel('Years')
axes[1, 0].set_ylabel('Million USD')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Sensitivity analysis
price_sensitivity = [results[f'Price_{p}_Cost_Base']['npv'] for p in ['Low', 'Base', 'High']]
cost_sensitivity = [results[f'Price_Base_Cost_{c}']['npv'] for c in ['Low', 'Base', 'High']]

x = np.arange(3)
width = 0.35

axes[1, 1].bar(x - width/2, price_sensitivity, width, label='Price Sensitivity')
axes[1, 1].bar(x + width/2, cost_sensitivity, width, label='Cost Sensitivity')
axes[1, 1].set_title('NPV Sensitivity to Price and Cost')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(['Low', 'Base', 'High'])
axes[1, 1].set_ylabel('NPV (Million USD)')
axes[1, 1].legend()
axes[1, 1].grid(True, axis='y')

plt.tight_layout()
plt.show()

# Print economic analysis results
print("="*60)
print("ECONOMIC ANALYSIS RESULTS")
print("="*60)

for scenario, result in results.items():
    print(f"{scenario}:")
    print(f"  NPV: ${result['npv']/1000:.2f} million")
    print(f"  IRR: {result['irr']*100:.1f}%")
    print(f"  Payback period: {result['payback_period']:.1f} years")
    print()

# Breakeven analysis
def breakeven_objective(oil_price):
    result = economic_model(oil_price, development_costs['Base'], production)
    return abs(result['npv'])

breakeven_result = minimize(breakeven_objective, x0=60, method='Nelder-Mead')
breakeven_price = breakeven_result.x[0] if breakeven_result.success else np.nan

print(f"Breakeven oil price: ${breakeven_price:.2f}/bbl")

# Save economic analysis results
economic_results = []
for scenario, result in results.items():
    economic_results.append({
        'Scenario': scenario,
        'NPV_Million_USD': result['npv'] / 1000,
        'IRR_Percent': result['irr'] * 100,
        'Payback_Period_Years': result['payback_period']
    })

economic_df = pd.DataFrame(economic_results)
economic_df.to_csv('economic_analysis_results.csv', index=False)
print("Economic analysis results saved to 'economic_analysis_results.csv'")

1. Oil Spill Analysis: Time series analysis, economic impact assessment, and statistical evaluation

2. Well Log Analysis: Petrophysical calculations, lithology identification, and net pay estimation

3. Production Analysis: Decline curve analysis, forecasting, and EUR estimation

4. Economic Analysis: Investment decision modeling, sensitivity analysis, and breakeven calculation